In [1]:
# Show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Imports.
import os

projectdir = '/Users/michielkleinnijenhuis'
dataset = 'HFK16w'

datadir = os.path.join(projectdir, dataset)

os.makedirs(datadir, exist_ok=True)
os.chdir(datadir)
f'working in directory: {os.path.abspath(".")}'

parameter_file = f'{dataset}.yml'
ims_filepath = f'{dataset}_shading_stitching.ims'  # f'{dataset}_shading_stitching_biasfield.ims'
image_in = ims_filepath


In [ ]:
from stapl3d.segmentation import zipping
from importlib import reload
reload(zipping)
zipp3r = zipping.Zipp3r(image_in, parameter_file, prefix=dataset)  # parameter file for segmentation hook
zipp3r.ids_labels = 'segm/labels_raw'
zipp3r.relabel()
zipp3r.estimate()


In [ ]:
zipp3r.view([0], images=['memb/prep'], labels=['segm/labels_raw', 'segm/labels_zip'])


In [ ]:
axlab = 'zyx'

zipp3r.margin = 64
zipp3r.n = {'z': 3, 'y': 3, 'x': 3}
# zipp3r.n = {'z': 2, 'y': 2, 'x': 2}
# zipp3r.n = {'z': 1, 'y': 1, 'x': 1}

zipp3r.zipstep = 'q'
zipp3r._axis = 0
zipp3r.zipstep = 'y'
zipp3r._axis = 1
zipp3r.zipstep = 'x'
zipp3r._axis = 2

zipp3r._set_blockmap(axislabels=axlab)

seam_idx = 0
seam_pairs = zipp3r.get_seam_pairs(seam_idx)
pairs = {i: pair for i, pair in enumerate(seam_pairs)}
pairs


In [ ]:
# Assembled block

from stapl3d.segmentation import zipping
from importlib import reload
reload(zipping)

ids = 'segm/labels_zip'

blockpair = [zipp3r._blocks[i] for i in pairs[0]]

blockset = zipping.Zipset(blockpair, zipp3r.zipstep, zipp3r.n)

blockset.read_margins(ids, imtype='Label')
blockset.assemble_dataset(ids, 'Label')
blockset.write_margins(ids)

# labelsets = zipp3r.get_boundary_labels(blockset)
# blockset.blocks[1].datasets[ods_labels].image.maxlabel


In [ ]:
import numpy as np
def _get_nmax(blocks, axis, margin, n_cap=10):
    """Calculate how many margin-blocks fit into the dataset."""

    sizes = []
    for al in 'zyx':  # TODO: 'zyx'
        for block in blocks:
            size = block.slices_region[al].stop - block.slices_region[al].start
            sizes.append(size)

        n_max = int(np.amin(np.array(sizes)) / margin)

    # TODO: change to variable size over dimensions
    #blockmargin = np.array([self.blockmargin[al] for al in 'zyx'])
#        n_max = np.array(np.array(sizes) / blockmargin, dtype='int')
    # n_max = np.minimum(np.array([n_cap]*3), n_max)

    n_max = int(np.amin(np.array(sizes)) / margin)
    n_max = min(n_cap, n_max)

    return n_max


In [ ]:
_get_nmax(blockset.blocks, axis=0, zipp3r.blockmargin, n_cap=10)


In [ ]:
block = blockset.blocks[1]

for al in zipp3r._axlab:
    if zipp3r.blockmargin[al]:
        n = int(block.shape[al] / zipp3r.blockmargin[al])
    else:
        n = 1
    print(al, n)

# block.datasets[ids].slices_blockfile


In [ ]:
capped = {al: False for al in 'zyx'}
all(capped.values())


In [ ]:
n_max = {}
for al in 'zyx':
    sizes = []
    for block in blockset.blocks:
        size = block.slices_region[al].stop - block.slices_region[al].start
        sizes.append(size)
    if zipp3r.blockmargin[al]:
        n_max[al] = int(block.shape[al] / zipp3r.blockmargin[al])
    else:
        n_max[al] = 1




In [ ]:
blockset_slices = blockset._get_slices(include_margin=False)
block = blockset.blocks[1]
slices = blockset_slices[1]
imtype = 'Label'
# blockset.read_margins(ids, imtype='Label')

props = dict(axlab=axlab, imtype=imtype, slices=slices)  # FIXME: dtype , dtype='int'
block.create_dataset(ids, **props)
block.path
block.datasets[ids].path


In [ ]:
block = blockset.blocks[1]
block.create_dataset(ids)
block.datasets[ids].path

In [ ]:
for al in axlab:
    slc = slice(
        blockset.full2block(block, al, slices[al].start),
        blockset.full2block(block, al, slices[al].stop),
    )
    block.datasets[ids].slices_blockfile[al] = slc


In [ ]:
ods_labels = 'segm/labels_zip'
axlab = 'zyx'
for block, labelset in zip(blockset.blocks, labelsets):
    seg = block.datasets[ods_labels].image
    fw_map = [True if l in labelset else False for l in range(0, seg.maxlabel + 1)]
    mask = np.array(fw_map)[seg.ds]

    block.create_dataset('mask', axlab=axlab)
    block_ds_in = block.datasets['mask']
    block_ds_in.dtype = mask.dtype
    block_ds_in.create_image(data=mask)
    block_ds_in.image.ds[:] = mask


In [ ]:
seg.maxlabel

In [ ]:

len(labelsets[1])
zipp3r.create_resegmentation_mask(blockset, labelsets)


In [ ]:

idss = {'mean': 'Image', 'memb/mean': 'Image'}
for ids, imtype in idss.items():
    blockset.read_margins(ids, imtype='Image')
    blockset.assemble_dataset(ids, imtype='Image')


In [ ]:
block.datasets['mean'].image.ds.shape
block.datasets['memb/mean'].image.ds.shape


In [ ]:
from stapl3d import blocks
splitt3r = blocks.Splitt3r(image_in, parameter_file, prefix=dataset)
splitt3r.run()

ims_filepath = f'{dataset}_shading_stitching.ims'  # f'{dataset}_shading_stitching_biasfield.ims'
image_in = ims_filepath

max_workers = 5  # NB: ACME is memory-intensive

from stapl3d.segmentation import enhance
enhanc3r = enhance.Enhanc3r(image_in, parameter_file, prefix=dataset, max_workers=max_workers)
enhanc3r.ACMEdir = os.environ.get('ACME')

if enhanc3r.ACMEdir:

    # Perform membrane enhancement.
    enhanc3r.run()

else:

    acme_filepath = f'{dataset}_shading_stitching_ACME.h5'

    # Download precomputed membrane enhancement.
    if not os.path.exists(acme_filepath):
        url = 'https://surfdrive.surf.nl/files/index.php/s/oQcxIocFBkaXwJe/download'
        urllib.request.urlretrieve(url, acme_filepath)

    # Split into blocks
    from stapl3d import blocks
    for ids in ['memb/preprocess', 'memb/planarity']:
        filepath_in = f'{acme_filepath}/{ids}'
        splitt3r = blocks.Splitt3r(filepath_in, parameter_file, prefix=dataset)
        splitt3r.volumes = {ids: {'output_ND': True}}  # override volumes from parameter file
        splitt3r.run()


In [2]:
from stapl3d.segmentation import segment
segment3r = segment.Segment3r(image_in, parameter_file, prefix=dataset)
segment3r.run()
# segment3r.view([2], images=['memb/mean'], labels=['segm/labels_raw'])  # , 'segm/labels', 'segm/labels_zip'


Running  segmentation:blockinfo in 64 jobs over 16 workers
Running  segmentation:estimate in 64 jobs over 16 workers
prep_dset took 3.278795 s for block B00007
prep_dset took 3.396240 s for block B00000
prep_dset took 3.946743 s for block B00002
prep_dset took 4.041285 s for block B00005
prep_dset took 4.006826 s for block B00006
prep_dset took 4.051975 s for block B00004
prep_dset took 4.101120 s for block B00008
prep_dset took 4.017288 s for block B00001
prep_dset took 4.038206 s for block B00015
prep_dset took 4.124699 s for block B00003
prep_dset took 4.828738 s for block B00010
prep_dset took 4.851858 s for block B00014
prep_dset took 4.906778 s for block B00012
prep_dset took 4.959867 s for block B00009
prep_dset took 4.956201 s for block B00013
prep_dset took 5.050965 s for block B00011
prep_dset took 3.734333 s for block B00016
prep_dset took 3.530109 s for block B00023
prep_dset took 3.832383 s for block B00024
prep_dset took 4.668385 s for block B00017
prep_dset took 4.214167

In [ ]:
segment3r.view([2], images=['memb/mean', 'memb/norm'], labels=['segm/labels_raw'])  # , 'segm/labels', 'segm/labels_zip'


block.create_dataset(ids, axlab=axlab)
block_ds_in = block.datasets[ids]
block_ds_in.dtype = data.dtype
block_ds_in.create_image(data=data)
block_ds_in.image.ds[:] = data

# Resegment the block.


In [ ]:
blockset.slices_read, blockset.slices_write


In [ ]:
# Write assembled block to blockset



In [ ]:
# zipsels = self._get_zipsels(len(blocks))
# for block, zipsel in zip(blocks, zipsels):

#     # Slice the margin from the concatenated block.
#     concat_slices = self._get_slices_concat(zipsel, axlab)
#     data_block = data[tuple(concat_slices.values())]

#     # slices = self._get_write_slices(block, zipsel, axlab)
#     slices = block._get_zip_slices(zipsel, self.n, axlab, include_margin=True)

#     block.create_dataset(ids, axlab=axlab, imtype=imtype)
#     block_ds = block.datasets[ids]
#     block_ds.dtype = 'int'
#     block_ds.create_image(f'{block.path}/{ids}')
#     block_ds.image.load()

#     for al in axlab:
#         slc = slice(
#             full2block(block, al, slices[al].start),
#             full2block(block, al, slices[al].stop),
#         )
#         dim = block_ds.image.axlab.index(al)
#         block_ds.image.slices[dim] = slc

#     update_vol(block_ds.image, data_block)


In [ ]:
data.shape, block.shape

In [ ]:
data.shape

In [ ]:

# im = block_ds_in.image



In [ ]:
vars(zipp3r._blocks[0])

In [ ]:
vars(block)


In [ ]:
ids = zipp3r.ods_labels
axlab = 'zyx'
imtype = 'Label'
block = zipp3r._blocks[0]
block.create_dataset(ids, axlab=axlab, imtype=imtype)
block_ds = block.datasets[ids]


In [ ]:
block_ds.dtype = 'int'
ds_path = f'{block.path}/{ids}'
block_ds.create_image(path=ds_path)


In [ ]:
block_ds.image.load()

In [ ]:
block_ds.image.ds

In [ ]:
block_ds.image.slices
block_ds.image.close()

In [ ]:
zipp3r.blockmargin
axlab = 'zyx'
zipsels = zipp3r._get_zipsels(2)


In [ ]:
axlab='zyx'
slices = zipp3r._get_slices_boundaries(zipsels[0], axlab)


In [ ]:
slices

In [ ]:
slices = {al: slice(None) for al in axlab}
for al, upper in zipsel.items():

    include_margin = False
    margin = self.blockmargin[al]
    b = margin if include_margin else 1
    slice_selector = {True: slice(-b, None), False: slice(None, b)}

    slices[al] = slice_selector[upper]


In [ ]:
zipp3r.margin = 64
zipp3r.n = 1

zipp3r.zipstep = 'q'
zipp3r._axis = 0
zipp3r.zipstep = 'x'
zipp3r._axis = 2

zipp3r.set_blockmap(axislabels='zyx')

seam_idx = 0
seam_pairs = zipp3r.get_seam_pairs(seam_idx)
pairs = {i: pair for i, pair in enumerate(seam_pairs)}
pairs


In [ ]:
blocks = [zipp3r._blocks[idx] for idx in [8, 9]]
segs, segs_ds, masks, masks_ds, mask = zipp3r.get_resegmentation_mask(blocks)


In [ ]:
# read the margin of the labelimage/zipmask blocks
segm_imgs, segm_dsets = zipp3r.read_images(blocks, zipp3r.ods_labels, 'Label')
mask_imgs, mask_dsets = zipp3r.read_images(blocks, zipp3r.ods_zipmask, 'Mask')


In [ ]:
zipp3r._blocks[9].datasets['segm/labels_zip'].slices_blockfile


In [ ]:
mn = zipp3r.margin * zipp3r.n
zipsels = zipp3r._get_zipsels(2)
zipsels

masks_ds.shape


In [ ]:
# block_slicer
slcs = [slice(None)] * 3
slc = {True: slice(None, mn), False: slice(-mn, -mn+1)}

slcs[2] = slc[0]
masks_ds[tuple(slcs)].shape


In [ ]:
t = [0,1,2,3,4,5]
t[slice(-1, None)]
t[slice(None, 1)]


In [ ]:
for al, upper in zipsel.items():
    if upper:
        slices[al] = slice(slices[al].stop - mn, slices[al].stop)
    else:
        slices[al] = slice(slices[al].start, slices[al].start + mn)


In [ ]:
# find the labels that are on the seam
# TODO: keep seam-labelset for next iteration
labelsets = zipp3r.get_labels(segm_dsets)
# check if there any labels in the combined set
if not set().union(*labelsets):
    pass
else:
    labelsets


In [ ]:
segm_imgs[1].path


In [ ]:

# create boolean forward maps
fw_maps = tuple([True if l in labelset else False for l in range(0, seg.maxlabel + 1)] for labelset, seg in zip(labelsets, segm_imgs))


In [ ]:
fw_maps

In [ ]:
import numpy as np
# create the resegmentation mask
rseg_dsets = tuple(np.array(fw_map)[segm_dset] for fw_map, segm_dset in zip(fw_maps, segm_dsets))


In [ ]:

# concatenate the margins of the block set
segm_dset = self.concat_images(segm_dsets, self._axis)
mask_dset = self.concat_images(mask_dsets, self._axis)
rseg_dset = self.concat_images(rseg_dsets, self._axis)

return segm_imgs, segm_dset, mask_imgs, mask_dset, rseg_dset


In [ ]:
#pairs = {0: [16, 24], 1: [23, 31]}
#pairs = {0: [16, 24], 1: [17, 25]}

#images = ['nucl/prep']
#labels = [zipp3r.ods_labels, zipp3r.ods_zipmask]
#images = None
#labels = [zipp3r.ods_zipmask]
images = ['nucl/prep']
labels = None
settings = {'clim': {'nucl/prep': [0, 30000]}}
zipp3r.view(pairs, images, labels, settings)


In [ ]:
# slices of margins into full dataset

axlab = 'zyx'
m = 64
n = 2
mn = m * n
zipstep = 'x'

idx = 9
# upper 
block = zipp3r._blocks[idx]
bslices = block.slices_region
slices = {al: slc for al, slc in bslices.items() if al in axlab}
slices[zipstep] = slice(slices[zipstep].stop - mn, slices[zipstep].stop)
slices

offsets = {al: slc.start for al, slc in bslices.items() if al in axlab}
offsets


In [ ]:
def foo(block, al, full_coord):
    offset = block.slices_region[al].start - block.slices_region_blockfile[al].start
    block_coord = full_coord - offset
    return block_coord

idx = 9
block = zipp3r._blocks[idx]
foo(block, 'y', 176)


In [ ]:
block.slices[al].start, block.slices_blockfile[al].start
block.slices_region[al].start, block.slices_region_blockfile[al].start


In [ ]:
al = 'x'
block._margins(
    fc=slices[al].start,
    fC=slices[al].stop,
    fullsize=zipp3r.fullsize[al],
    blocksize=zipp3r.blocksize[al],
    margin=zipp3r.blockmargin[al],
    shift_final_block_inward=False,
)


In [ ]:
# lower
idx = 12
block = zipp3r._blocks[idx]
bslices = block.slices_region
slices = {al: slc for al, slc in bslices.items() if al in axlab}
slices[zipstep] = slice(slices[zipstep].start, slices[zipstep].start + mn)
slices

offsets = {al: slc.start for al, slc in bslices.items() if al in axlab}
offsets


In [ ]:
foo(block, slices[al].start)


In [ ]:
block.slices[al].start, block.slices_blockfile[al].start


In [ ]:
al = 'x'
block._margins(
    slices[al].start, slices[al].stop,
    fullsize=zipp3r.fullsize[al],
    blocksize=zipp3r.blocksize[al],
    margin=zipp3r.blockmargin[al],
    shift_final_block_inward=False,
)


In [ ]:
zipp3r.fullsize

In [ ]:
block0 = zipp3r._blocks[0]
block = zipp3r._blocks[idx]
ids = 'segm/labels_zip'

block.create_dataset(ids, axlab=axlab, slices=slices)
block_ds = block.datasets[ids]

for al in 'zyx':
    slc = slice(
        block_ds.slices[al].start - offsets[al],
        block_ds.slices[al].stop - offsets[al],
    )
    block_ds.slices_blockfile[al] = slc

# block.slices, block.slices_region, block.slices_blockfile, block.slices_region_blockfile
# block_ds.slices, block_ds.slices_blockfile
#block_ds.slices_region, block_ds.slices_region_blockfile
# block_ds.shape


In [ ]:
block_ds.dtype = 'int'
block_ds.create_image()
block_ds.read(from_block=True, padded=False)


In [ ]:
import numpy as np
spacing = [block_ds.elsize[al] for al in block_ds.axlab if al in 'xyz']
starts = [slc.start for al, slc in block.slices.items() if al in 'xyz']
origin = [sp * st for sp, st in zip(spacing, starts)]
spacing = spacing or [1, 1, 1]
origin = origin or [0, 0, 0]
from stapl3d.preprocessing.registration import get_affine
affine = get_affine([0, 0, 0], spacing, origin)
affine = np.copy(affine)
print(affine, affine2)


In [ ]:
240*elsize[2]

In [ ]:
import napari

In [ ]:
affine3 = np.array(affine)
affine3[2,3] = 0
affine2 = np.array(affine)
affine2[2,3] = affine[2,3] + 79.7050568181818
viewer = napari.Viewer()
elsize = [block_ds.elsize[al] for al in 'zyx']
viewer.add_labels(block_ds.image.ds, name='foo', affine=affine)
viewer.add_labels(block_ds.image.ds, name='foo', affine=affine2)
viewer.add_labels(block_ds.image.ds, name='foo', affine=affine3)


In [ ]:
block_ds.image.ds.shape


In [ ]:
24*0.33

In [ ]:
block_ds.shape

In [ ]:
name = f'{block.id}_{ids}'
data = block_ds.image.ds
elsize = [block_ds.elsize[al] for al in block_ds.axlab]
name, data.shape, elsize


In [ ]:
block_ds.image.shape

In [ ]:
import numpy as np

#zipp3r.zipsteps = 'zy'
#zipp3r.seams = [0]

#zipp3r.estimate()


In [ ]:
zipp3r.margin = 64
zipp3r.n = 2
zipp3r.zipstep = 'x'
zipp3r._axis = 2


In [ ]:
zipp3r.ods_labels

In [ ]:
block_ds.slices_margin_blockfile = {al: slc for al, slc in block.slices_margin_blockfile.items() if al in axlab}
block_ds.slices_margin_blockfile


In [ ]:
block0 = zipp3r._blocks[block_idxs[0]]

for block_idx in block_idxs:

    block = self._blocks[block_idx]

    for ids in images + labels:

        block.create_dataset(ids)
        block_ds = block.datasets[ids]
        block_ds.read(from_block=True, read_margin=True)  # TODO: read only slices_blockfile_margins
        elsize = [block_ds.elsize[al] for al in block_ds.axlab]

        name = f'{block.id}_{ids}'
        data = block_ds.image.ds

        spacing = [block_ds.elsize[al] for al in block_ds.axlab if al in 'xyz']
        starts = [slc.start for al, slc in block.slices.items() if al in 'xyz']
        origin = [sp * st for sp, st in zip(spacing, starts)]
        spacing = spacing or [1, 1, 1]
        origin = origin or [0, 0, 0]
        from stapl3d.preprocessing.registration import get_affine
        affine = get_affine([0, 0, 0], spacing, origin)
        affine = np.copy(affine)

        if ids in images:
            self.viewer.add_image(data, name=name, scale=elsize, affine=affine)
            clim = self.viewer.layers[f'{block0.id}_{ids}'].contrast_limits
            self.viewer.layers[f'{block.id}_{ids}'].contrast_limits = clim
        else:
            self.viewer.add_labels(data, name=name, scale=elsize, affine=affine)

axlab = block_ds.axlab

self.viewer.dims.axis_labels = [al for al in axlab]


In [ ]:
zipp3r.zipstep = 'x'
zipp3r._axis = 2
zipp3r.margin = 64
zipp3r.maxlabel = 1000000
pair = [0, 1]
zipp3r.j = 0
blocks = [zipp3r._blocks[idx] for idx in pair]
n_max = zipp3r._get_nmax(blocks, zipp3r._axis, zipp3r.margin)  # TODO: n_max as ndim
print(n_max)

#zipp3r.process_pair(0, blocks, n_max)


In [ ]:
zipp3r.zipstep


In [ ]:
zipp3r.n = min(n_max, 3)  # number of blockmargins

segm_imgs, segm_dsets = zipp3r.read_images(blocks, zipp3r.ods_labels, 'Label')
mask_imgs, mask_dsets = zipp3r.read_images(blocks, zipp3r.ods_zipmask, 'Mask')
labelsets = zipp3r.get_labels(segm_dsets)


# create boolean forward maps
fw_maps = tuple([True if l in labelset else False for l in range(0, seg.maxlabel + 1)] for labelset, seg in zip(labelsets, segm_imgs))
# create the resegmentation mask
rseg_dsets = tuple(np.array(fw_map)[segm_dset] for fw_map, segm_dset in zip(fw_maps, segm_dsets))

# concatenate the margins of the block set
segm_dset = zipp3r.concat_images(segm_dsets, zipp3r._axis)
mask_dset = zipp3r.concat_images(mask_dsets, zipp3r._axis)
rseg_dset = zipp3r.concat_images(rseg_dsets, zipp3r._axis)


In [ ]:
np.sum(segm_dsets[0]), np.sum(segm_dsets[1])
np.sum(fw_maps[0]), np.sum(fw_maps[1])
np.sum(rseg_dsets[0]), np.sum(rseg_dsets[1])


In [ ]:
blocks[1].slices, blocks[1].slices_blockfile

In [ ]:
spacing = [es for al, es in elsize.items() if al in 'xyz']
starts = [slc.start for al, slc in self.slices.items() if al in 'xyz']
origin = [sp * st for sp, st in zip(spacing, starts)]
spacing = spacing or [1, 1, 1]
origin = origin or [0, 0, 0]
self.affine = get_affine([0, 0, 0], spacing, origin)


In [ ]:
zipp3r.view_pair()


In [ ]:
import napari
viewer = napari.Viewer()
viewer.add_labels(segm_dsets[0])
viewer.add_labels(segm_dsets[1])


In [ ]:
#fw_maps[1]
np.unique(segm_dsets[0])


In [ ]:
segm_imgs[0].maxlabel, segm_imgs[1].maxlabel

In [ ]:
len(fw_maps[1]), np.max(segm_dsets[1])


In [ ]:
segs, segs_ds, masks, masks_ds, mask = zipp3r.get_resegmentation_mask(blocks)
np.sum(mask)

In [ ]:
segm_dset.shape

In [ ]:
import numpy as np
np.sum(rseg_dset)


In [ ]:
axlab = 'zyx'  # FIXME: assuming zyx order
zipp3r.set_blockmap(axislabels=axlab)
zipp3r.blockmap
zipp3r.zipstep = 'y'
zipp3r._axis = zipp3r.zipsteps.index(zipp3r.zipstep)
seam_idx = 0
seam_pairs = zipp3r.get_seam_pairs(seam_idx)
seam_pairs
list(seam_pairs[seam_idx])


In [ ]:
import numpy as np

zipp3r.zipsteps = 'x'
zipp3r.seams = [0]

"""
zipp3r.zipstep = 'y'
zipp3r._axis = zipp3r.zipsteps.index(zipp3r.zipstep)


offsets = [0, 1]
seamgrid_shape = np.maximum(np.array(zipp3r.blockmap.shape) - 1, np.array([1, 1, 1]))

for offset in offsets:

    zipp3r._seams = []
    for seam_idx in range(offset, seamgrid_shape[zipp3r._axis], 2):
        zipp3r._seams.append(seam_idx)

zipp3r._seams
"""

zipp3r.estimate()


In [ ]:
seamgrid_shape
zipp3r._axis


In [ ]:
images = ['nucl/prep', 'memb/prep']
labels = ['segm/labels', 'segm/labels_zip', 'segm/labels_zipmask']
zipp3r.view(list(seam_pairs[0]), images, labels)


In [ ]:
import napari

In [ ]:
import numpy as np
self = zipp3r
seam_idx = 0
#zipp3r.get_seam_pairs(0)
seamgrid_shape = np.maximum(np.array(self.blockmap.shape) - 1, np.array([1, 1, 1]))
seamnumbers = np.unravel_index(seam_idx, seamgrid_shape)
seamnumbers


In [ ]:
# Slice the blockmap to get the block indices of the blocks around the seam.
if self.zipstep == 'q':
    slcs = [slice(seamnumbers[d], seamnumbers[d] + 2) for d in range(3)]
    sh = (1, 4)  # single quad as list of len 4?
else:
    axis = 'zyx'.index(self.zipstep)  # FIXME: assuming zyx order
    slcs = [slice(None) for d in range(3)]
    slcs[axis] = slice(seamnumbers[0], seamnumbers[0] + 2)
    sh = (-1, 2)  # list of pairs


In [ ]:
pairs = np.squeeze(self.blockmap[tuple(slcs)])
# FIXME: assumes zyx order
#newaxes = {'x': [0, 1, 2], 'y': [2, 0, 1], 'z': [2, 1, 0], 'q': [0, 1, 2]}
#pairs = np.moveaxis(pairs, [0, 1, 2], newaxes)
axis = self._axis if self.zipstep in 'zyx' else 0
pairs = np.moveaxis(pairs, [0, 1, 2], [2, 1, 0])
pairs = np.reshape(pairs, sh)


In [ ]:
pairs

In [ ]:
# ax: operate over these axes, # tp: transpose axes, # sh: reshape
tps = {'x': [0, 1], 'y': [1, 0], 'z': []}  # TODO: determine tp_z

# Slice the blockmap to get the block indices of the blocks around the seam.
if self.zipstep == 'q':
    slcs = [slice(seamnumbers[d], seamnumbers[d] + 2) for d in range(3)]
    tp = [0, 1]
    sh = (1, 4)  # single quad as list of len 4?
else:
    axis = 'zyx'.index(self.zipstep)  # FIXME: assuming zyx order
    slcs = [slice(None) for d in range(3)]
    slcs[axis] = slice(seamnumbers[0], seamnumbers[0] + 2)
    tp = tps[self.zipstep]
    sh = (-1, 2)  # list of pairs

pairs = np.squeeze(self.blockmap[tuple(slcs)])
pairs



In [ ]:
# roll axis until it is at the back

pairs.shape
#np.transpose(pairs, [2, 1, 0])


In [ ]:
help(np.moveaxis)

In [ ]:
pairs = np.reshape(, sh)
pairs

## Parallelization

We have specified the shape of the processing blocks in the parameter file. Usually we would opt for a blocksize of ~100-200 million voxels; now we chose a blocksize in xy of 176 for 64 blocks of ~6M voxels. We keep the margin similar to what we set for big datasets as reducing it may hinder adequate analysis.

In [ ]:
yprint(cfg['blocks']['blockinfo'])


The full anatomy of the blocked processing can now be loaded through the blocker object.


In [ ]:
from stapl3d import blocks
block3r = blocks.Block3r(image_in, parameter_file, prefix=dataset)
print(block3r)


In initializing the *block3r* object, the sizes for the zyxct-dimensions were read from the input data and the dimensions that were specified in the configuration file for blocksize were substituted to determine the 5D-blocksize. 

For segmentation, we use a weighted sum of the membrane channels (ch3, ch5, ch6, ch7). The weights [0.5, 0.5, 1.0, 1.0] work well for this data. We set the name and internal .h5 path to 'memb/mean'.
We have specified this in the parameter file HFK16w.yml:

In [ ]:
yprint(cfg['splitter']['split']['volumes'])


The above indicates that, in addition to the membrane sum, we generate a mean over all channels (used for generating masks). Importantly, we specify that we want to output channel 0 (DAPI), because we will use it to create a nuclear mask. 

Now we are ready to call the function that computes the membrane mean, and splits the data into blocks at the same time.

In [ ]:
splitt3r = blocks.Splitt3r(image_in, parameter_file, prefix=dataset)
splitt3r.run()


Datablocks are written to the *HFK16w/blocks/* directory and are postfixed by the numeric ID of the block HFK16w/blocks/HFK16w_**B{b:05}**.h5.

These are some of the files that were generated:

In [ ]:
from glob import glob

filelist = glob(os.path.join(os.path.abspath('.'), 'blocks', f'{dataset}_*.h5'))
filelist.sort()

f'Number of blocks: {len(filelist)}'
filelist[:5]


The resulting hdf5 files have three datasets named according to 'outputvolumes' entries in the cfg['splitter']['split'] specification, i.e. they have the following internal file structure:
- <...>.h5/mean
- <...>.h5/chan/ch00
- <...>.h5/memb/mean

It can be inspected and listed with the help of h5py:

In [ ]:
import h5py

def extract_node_names(name, node):
    if isinstance(node, h5py.Dataset):
        nodes.append(name)
    return None

nodes = []
with h5py.File(filelist[0], 'r') as f:
    f.visititems(extract_node_names)
    pprint({'dataset names': nodes})
    idx = 0
    print(f'dataset {nodes[idx]} properties: ', f[nodes[idx]])
    print(f'dataset {nodes[idx]} resolution: ', f[nodes[idx]].attrs['element_size_um'])
    print(f'dataset {nodes[idx]} axes labels: ', f[nodes[idx]].attrs['DIMENSION_LABELS'])


Naturally, we also need to visually inspect the resulting averaged volumes. We can use the napari viewer method provided in the *splitt3r* object. We limit to the first 42 blocks and pick the mean membrane channel for demonstration. 

In [ ]:
idxs = list(range(42))  # block indices
images = ['memb/mean']
viewer_settings = {
    'title': 'STAPL3D splitt3r demo',
    'crosshairs': [int(splitt3r.fullsize[dim] / 2) for dim in 'zyx'],
    'axes_visible': False,
    'clim': [0, 5000],
}

splitt3r.view(input=idxs, images=images, settings=viewer_settings)


To get a grip on how the dataset is layed out in blocks, we can alternate the colormaps of the blocks.

In [ ]:
# Alternate colormaps.
cmaps = ['cyan', 'magenta', 'yellow']
for i, lay in enumerate(splitt3r.viewer.layers):
    lay.colormap = cmaps[i % len(cmaps)]
lay.colormap = 'gray'


## Membrane enhancement

Before segmentation, we perform membrane enhancement.

For the demo we do not want to be dependent on the third-party [ACME](https://wiki.med.harvard.edu/SysBio/Megason/ACME) software and provide the output that otherwise results from the ACME procedure. We split it into blocks, and write it as separate datasets in the same files as the channel data.

Alternatively, if you have ACME installed and want to run it, set an `ACME` path environment variable or point `ACMEdir` to the directory with the binaries.

In [ ]:
ims_filepath = f'{dataset}_shading_stitching.ims'  # f'{dataset}_shading_stitching_biasfield.ims'
image_in = ims_filepath

max_workers = 5  # NB: ACME is memory-intensive

from stapl3d.segmentation import enhance
enhanc3r = enhance.Enhanc3r(image_in, parameter_file, prefix=dataset, max_workers=max_workers)
enhanc3r.ACMEdir = os.environ.get('ACME')

if enhanc3r.ACMEdir:

    # Perform membrane enhancement.
    enhanc3r.run()

else:

    acme_filepath = f'{dataset}_shading_stitching_ACME.h5'

    # Download precomputed membrane enhancement.
    if not os.path.exists(acme_filepath):
        url = 'https://surfdrive.surf.nl/files/index.php/s/oQcxIocFBkaXwJe/download'
        urllib.request.urlretrieve(url, acme_filepath)

    # Split into blocks
    from stapl3d import blocks
    for ids in ['memb/preprocess', 'memb/planarity']:
        filepath_in = f'{acme_filepath}/{ids}'
        splitt3r = blocks.Splitt3r(filepath_in, parameter_file, prefix=dataset)
        splitt3r.volumes = {ids: {'output_ND': True}}  # override volumes from parameter file
        splitt3r.run()


We vizualize the membrane-enhanced volume with napari.

In [ ]:
# Initialize viewer.
idxs = list(range(42))  # block indices
images = ['memb/planarity']
viewer_settings = {
    'title': 'STAPL3D enhanc3r demo',
    'crosshairs': [int(enhanc3r.blocksize[dim] / 2) for dim in 'zyx'],
    'axes_visible': False,
    'clim': [0, 0.05],
}

enhanc3r.view(input=idxs, images=images, settings=viewer_settings)


## Segmentation

The segmentation is parallelized over the blocks we just created. Each of the 64 files is processed seperately.
The segmentation routine is associated with a fair amount of steps and parameters. This list all the parameters specified in the yml-file.

In [ ]:
yprint(cfg['segmentation']['estimate'])  # TODO: need to preserve order of print


The blocks will processed in order according to the steps defined in the parameter file. Operations are listed below and step names have to be prefixed with these keywords.
- *prep*: filtering of volumes
- *mask*: compartment mask creation
- *combine*: mask combination
- *seed*: seed generation
- *segment*: watershed segmentation
- *filter*: size filtering and label masking

For your own data, it is advised to start with tuning the following parameters to optimize segmentation:
- mask_memb : threshold
- mask_nucl : sauvola : window_size
- mask_nucl : sauvola : threshold
- mask_nucl : sauvola : absmin
- seeds : peaks : window
- seeds : peaks : window
- segment : watershed : compactness
- prep_memb : filter : sigma
  

Next we calculate the segments for each block. Segmentation time of single block is in the minutes-range. The 106 x 240 x 240 blocksize (including the margin) will take ~1GB of memory per process. Please set the number of processes so that you will stay within RAM. `max_workers = 8` would be a fairly safe bet for modern systems; `max_workers = 0` results in using all available processors.

In [ ]:
from stapl3d.segmentation import segment

max_workers = 0

segment3r = segment.Segment3r(image_in, parameter_file, prefix=dataset, max_workers=max_workers)
segment3r.run()


Report pages (pdf) have been written to the *HFK16w/blocks/* directory. Let's look at one of them:

In [ ]:
block_idx = 1
# Get the outputpaths of the 'estimate' method for a block.
_, opaths = segment3r.fill_paths('estimate', reps={'b': block_idx})
# (Re)generate the report from the data and plot inline.
segment3r.report(outputpath=None, ioff=False, outputs=opaths)


From top to bottom, images are show for:
 - the smoothed DAPI and mean membrane channels
 - the nuclear mask and the membrane mask
 - the combined mask with detected peaks and overlaid on the distance transform image
 - the first and the final watershed results


We use the 'labels' argument to visualize masks and labels in napari. First, this overlays the membrane mask with the planarity volume for the top row of blocks.

In [ ]:
idxs = list(range(8))  # block indices
images = ['memb/planarity']
labels = ['nucl/mask']

viewer_settings = {
    'title': 'STAPL3D segment3r demo',
    'crosshairs': [int(segment3r.blocksize[dim] / 2) for dim in 'zyx'],
    'axes_visible': False,
    'clim': {'memb/planarity': [0, 0.05]},
    'opacity': {'nucl/mask': 0.5},
}

segment3r.view(input=idxs, images=images, labels=labels, settings=viewer_settings)


Finally, we look at the extracted segments for the block we view as a pdf report.

In [ ]:
images = ['nucl/prep']
labels = ['segm/labels']

viewer_settings = {
    'title': 'STAPL3D segment3r demo',
    'crosshairs': [int(segment3r.blocksize[dim] / 2) for dim in 'zyx'],
    'axes_visible': False,
    'clim': {'nucl/prep': [0, 20000]},
    'opacity': {'segm/labels': 0.8},
}

segment3r.view(input=block_idx, images=images, labels=labels, settings=viewer_settings)


# zipping

Having parallelized the segmentation process for increased analysis speed and reduced memory footprint, the need arises to reassemble the blocks into a final combined segmentation volume without seams at the block boundaries. These seams are a consequence of trivial parallelization in processing the individual blocks (i.e. without communication between the processes). They manifest through partial cells lying on the block boundaries that have been assigned different labels in different blocks. Importantly, these doubly segmented cells may not perfectly match up over the boundary.

This can be demonstrated by loading the segmentation of the top two rows of blocks:

In [ ]:
segment3r.view(input=list(range(16)), images=images, labels=labels, settings=viewer_settings)


These block-boundary-segments need to be resegmented in order to complete the accurate segmentation of the full dataset. We refer to this correct reassembly of the datablocks as ‘zipping’. In short, it consists of identifying the segments lying on the boundaries, removing them, and resegmenting that space. We aimed to design the procedure such that it requires minimal computational resources and expertise (fast, with a low memory footprint, and without the need for communication between processes).

There is a one-liner for computing all the steps in the zip:
```
zipp3r.run()
```
which combines these two steps:
```
zipp3r.relabel()
zipp3r.estimate()
```

For this demo, we will be much more verbose to illustrate the zipping process. 


We define a convenience function that merges datablocks into a single volume and returns a single z-plane for display.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

def slice_dset(filepath, slc=20):
    """Return sliced data."""

    im = Image(filepath)
    im.load()
    im.slices[im.axlab.index('z')] = slice(slc, slc + 1, 1)
    data = im.slice_dataset()
    im.close()

    return data

def merge_and_slice_dset(merg3r, ids, slc=20):
    """Merge volume and return sliced data."""

    # Run the block merge.
    merg3r.vols = [list(merg3r.volumes.keys()).index(ids)]  # select volume by index
    merg3r.run()

    # Get a slice of the merged data.
    opaths = merg3r.outputpaths['merge']
    reps = {'a': ids, 'A': ids.replace('/', '-')}  # insert volume name for a/A
    opaths = merg3r._prep_paths(opaths, reps=reps)
    data = slice_dset(opaths['volume'], slc)

    return data


For plotting labels, we define a label shuffling function.

In [ ]:
import numpy as np
from skimage.segmentation import relabel_sequential
from random import shuffle
from skimage.color import label2rgb

def shuffle_labels(labels):
    """Shuffle labels in a volume."""

    labels = relabel_sequential(labels)[0]
    ulabels = np.unique(labels[:])[1:]
    relabeled = [l for l in range(0, len(ulabels))]
    shuffle(relabeled)
    labels = np.array([0] + relabeled)[labels]

    return labels


The merger object is prepped for merging 4 output volumes defined in the parameter file.

In [ ]:
from stapl3d import blocks
merg3r = blocks.Merg3r(image_in, parameter_file, prefix=dataset)
merg3r.volumes


Let's check with the membrane mean blocks. This should output an image of 1408 x 1408.

In [ ]:
img = merge_and_slice_dset(merg3r, ids='memb/mean')


In [ ]:
f = plt.figure(figsize=(8, 8))
plt.imshow(img, cmap='gray', vmax=5000)
plt.show()


In the same way, we can show the labels with the seams before zipping.

In [ ]:
labels = shuffle_labels(merge_and_slice_dset(merg3r, ids='segm/labels'))


In [ ]:
f = plt.figure(figsize=(8, 8))
plt.imshow(label2rgb(labels))
plt.show()


Initialize the zipp3r.


In [ ]:
from stapl3d.segmentation import zipping
#from importlib import reload
#reload(zipping)
zipp3r = zipping.Zipp3r(image_in, parameter_file, prefix=dataset)


We first perform a sequential relabeling of all the blocks to make each label unique and save it to a new dataset 'segm/labels_zip'. Also a volume 'segm/labels_zipmask' is created to track the area that is resegmented.


In [ ]:
zipp3r.relabel()


Next we set up the zipping estimation.

In [ ]:
import numpy as np

# write maxlabels to file
arglist = zipp3r._prep_step('estimate')
np.array(zipp3r._gather_maxlabels(zipp3r.ods_labels))


In the zipping procedure, we employ an order such that no blocks are handled concurrently. First, blocks with overlap in the Y-dimension are processed (odd and even zip-lines separately); then X-ziplines; then the corners where four datablocks overlap are resegmented. For demo purpose, we keep track of the output for each step and store it in `imgs`.

In [ ]:
imgs = []

# ziplines
for axis in [1, 2]:
    for offset in [0, 1]:
        n_seams = zipp3r._seamgrid.shape[axis - 1]
        zipp3r.compute_zip_step(
            axis=axis,
            starts=[offset, 0],
            stops=[n_seams, 1],
            steps=[2, 2],
            )
        _ = zipp3r._gather_maxlabels(zipp3r.ods_labels)
        imgs.append(merge_and_slice_dset(merg3r, 'segm/labels_zip'))

# zipquads
for start_y in [0, 1]:
    for start_x in [0, 1]:
        zipp3r.compute_zip_step(
            axis=0,
            starts=[start_y, start_x],
            stops=zipp3r._seamgrid.shape,
            steps=[2, 2],
            )
        _ = zipp3r._gather_maxlabels(zipp3r.ods_labels)
        imgs.append(merge_and_slice_dset(merg3r, 'segm/labels_zip'))


In [ ]:
f, axs = plt.subplots(2, 4, figsize=(24, 12))
for img, ax in zip(imgs, axs.flat):
    ax.imshow(img)
plt.show()


Newly processed zip-lines are assigned high labels indicated in yellow of the viridis colormap, nicely demonstrating the zipping process.

<!---The zip-lines still have seams in the places where they intersect. Next we process zip-quads, in which the segments on these intersections are resegmented to finish the zip.-->

Now, we compare the labels before and after the zip:

In [ ]:
labels_zipped = shuffle_labels(merge_and_slice_dset(merg3r, ids='segm/labels_zip'))


In [ ]:
f, axs = plt.subplots(1, 2, figsize=(16, 32))
for img, ax in zip([labels, labels_zipped], axs.flat):
    ax.imshow(label2rgb(shuffle_labels(img)))
plt.show()


View the zip result with napari:

In [ ]:
idss = ['segm/labels', 'segm/labels_zip']

merg3r = blocks.Merg3r(image_in, parameter_file, prefix=dataset)
merg3r.vols = [list(merg3r.volumes.keys()).index(ids) for ids in idss]
merg3r.run()

viewer_settings = {
    'title': 'STAPL3D merg3r demo',
    'crosshairs': [int(merg3r.blocksize[dim] / 2) for dim in 'zyx'],
    'axes_visible': False,
    'clims': [0, 1],
    'opacity': 1,
}

filepath = merg3r.outputpaths['postprocess']['aggregate']
merg3r.view(input=filepath, images=[], labels=idss, settings=viewer_settings)

# NOTE: some new seams are created on the margins because the blocks of this demo are too small


## Compartmental segmentation

In STAPL-3D, we use rich multidimensional data to obtain a robust segmentation. We can also use the information we have to perform subcellular segmentation. Here, we split segments in nucleus and membrane subsegments such that we can specifically extract intensities from the appropriate voxels for the type of staining (nuclear or membranal). In addition, the subsegmentation opens up possibilities for defining compound features that inform on internal cell structure.


In [ ]:
from stapl3d.segmentation import segment

subsegment3r = segment.Subsegment3r(image_in, parameter_file, prefix=dataset)
subsegment3r.run()


In [ ]:
vols = [f'segm/labels_{vol}' for vol in ['full', 'nucl', 'memb']]
merg3r = blocks.Merg3r(image_in, parameter_file, prefix=dataset)
merg3r.volumes = {vol: {'format': 'h5', 'suffix': None, 'is_labelimage': True} for vol in vols}
merg3r._init_paths_merger()
merg3r.run()

viewer_settings = {
    'title': 'STAPL3D merg3r demo',
    'crosshairs': [int(merg3r.blocksize[dim] / 2) for dim in 'zyx'],
    'axes_visible': False,
    'clims': [0, 1],
    'opacity': 1,
}

filepath = merg3r.outputpaths['postprocess']['aggregate']
merg3r.view(input=filepath, images=[], labels=vols, settings=viewer_settings)


In [ ]:
filepath